# POS-Tagging mit maschinellen Lernverfahren

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## Trainings- und Testdaten

Trainings (`_train`) und Testdaten (`_dev`) laden. Da die TSV-Tabellen keine Kopfzeile haben, müssen wir sinnvolle Spaltennamen selbst festlegen. Wichtig ist dabei, dass Anführungszeichen als normale Zeichen gelesen (`quoting=3`) und `null`, `N/A` usw. nicht als undefinierte Werte interpretiert werden (sehr unintuitiv mit `keep_default_na=False`).

In [ ]:
header = ("sent", "tok", "word", "pos", "lemma", "morph")
train = pd.read_csv("data/tiger2_train.tsv.gz", sep="\t", names=header, quoting=3, keep_default_na=False)
test = pd.read_csv("data/tiger2_dev.tsv.gz",sep="\t",names=header, quoting=3, keep_default_na=False)

In [ ]:
train

Wir benötigen nur die ersten 4 Spalten (`lemma` und `morph` können wir später für andere Aufgaben nutzen).

In [ ]:
train = train.iloc[:, 0:4]
test = test.iloc[:, 0:4]

Strings werden in Pandas als Arrays vom Typ `object` eingelesen.  Wir können sie explizit in spezielle `StringArray`s konvertieren, die aber [momentan wohl noch keine besonderen Performance-Vorteile](https://pandas.pydata.org/docs/user_guide/text.html) bieten.

In [ ]:
train.word = train.word.astype('string')
train.pos  = train.pos.astype('string')
test.word  = test.word.astype('string')
test.pos   = test.pos.astype('string')
train.dtypes

## Unigramm-Tagger (ohne Kontext)

Wir implementieren zunächst einen Unigramm-Tagger, der nur auf Wortformen arbeitet und keine Kontextinformation hinzuzieht. Das lässt sich besonders einfach mit der Pandas-Repräsentation der Trainingsdaten umsetzen.

Das naheliegendste Merkmal sind die Wortformen selbst, für die wir ein Dummy Coding (_one-hot encoding_) erstellen müssen.  Dies lässt sich direkt mit dem `OneHotEncoder` erstellen (oder dem `DictVectorizer`, der aber wesentlich mehr Overhead produziert).  Ein besonders einfacher und flexibler Ansatz ist ein `CountVectorizer`, der jeweils nur ein Token als Eingabe bekommt und viele weitere Optionen anbietet.  Wir vergleichen hier die Verarbeitungsgeschwindigkeit aller drei Varianten.

**1) OneHotEncoder:** Erwartet Listen oder Tupeln von Merkmalswerten, die jeweils in ein _one-hot encoding_ überführt werden. Hier müssen wir die Wörter also in Tupel der Länge 1 transformieren. Die Option `handle_unknown` muss auf `'ignore'` gesetzt werden, damit unbekannte Wortformen in den Testdaten keinen Fehler werfen. Mit `min_frequency` und `'infrequent_if_exist'` kann eine OOV-Kodierung implementiert werden. Wir setzen hier eine Schwellenwert von $f \ge 5$ an, damit nur halbwegs zuverlässige Lexikoninformation gelernt wird.

In [ ]:
wf_vectorizer = OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=5)
%time X_wf = wf_vectorizer.fit_transform([(x,) for x in train.word])
%time testX_wf = wf_vectorizer.transform([(x,) for x in test.word])
X_wf.shape

**2) DictVectorizer:** Hier stehen keine Optionen zur Auswahl, insbesondere könnte ein Schwellenwert wie $f\ge 5$ nur als zusätzliche Transformation der Merkmalsmatrix umgesetzt werden. Obwohl als Eingabe eine Liste von Dictionaries erstellt werden muss, ist dieser Ansatz schneller als der OneHotEncoder.

In [ ]:
wf_vectorizer = DictVectorizer()
%time X_wf = wf_vectorizer.fit_transform([{"word": x} for x in train.word])
%time testX_wf = wf_vectorizer.transform([{"word": x} for x in test.word])
X_wf.shape

**3) CountVectorizer:** Wir können das _one-hot encoding_ auch mit einem `CountVectorizer` erzeugen, der jede Wortform als ein einzelnes Token behandelt (was durch einen geeigneten Custom-Tokenizer sichergestellt werden muss). Mit `binary=True` könnten wir auch explizit erzwingen, dass eine binäre Matrix erzeugt wird. Der Schwellenwert $f\ge 5$ lässt sich leicht mit `min_df` anwenden (in der Binärmatrix ist ja $f = \mathit{df}$); allerdings werden hier die OOV nicht explizit repräsentiert (sondern durch einen $\mathbf{0}$-Vektor) und müssen implizit von dem maschinellen Lernverfahren gelernt werden. Dieser Ansatz ist zwar etwas langsamer als der `DictVectorizer`, wegen seiner Flexibilität aber vorzuziehen.

In [ ]:
wf_vectorizer = CountVectorizer(tokenizer=lambda x: (x,), lowercase=False, min_df=5)
%time X_wf = wf_vectorizer.fit_transform(train.word)
%time testX_wf = wf_vectorizer.transform(test.word)
X_wf.shape

Wir können nun ein erstes Lernexperiment mit einer linearen SVM durchführen (ohne Optimierung der Meta-Parameter).

In [ ]:
%%time
clf = LinearSVC()
clf.fit(X_wf, train.pos)

Evaluation auf den Testdaten ergibt schon eine ganz passable Genauigkeit. Ein Vergleich mit den Trainingsdaten zeigt, dass die SVM kaum übertrainiert ist (**Frage:** Was könnte der Grund dafür sein?).

In [ ]:
predicted = clf.predict(testX_wf)
print(accuracy_score(predicted, test.pos))

print(clf.score(X_wf, train.pos))

### Fehleranalyse

Der beste Ausgangspunkt für eine gezielte Optimierung der Lernergebnisse ist oft eine detaillierte Fehleranalyse. Als ersten Schritte berechnen wir Precision und Recall separat für jede Kategorie, d.h. jedes POS-Tag:

In [ ]:
print(classification_report(test.pos, predicted, zero_division=0))

Schlechte Ergebnisse bei offenen Wortklassen könnten zu einem erheblichen Teil auf unbekannte Wörter zurückzuführen sein, für die immer die gleiche Wortart geraten wird. Bei geschlossenen Wortklassen deuten sie darauf hin, dass Kontextinformation zur Disambiguierung notwendig wäre. So etwa bei `PTKVZ` (abgetrennte Verbpartikel, z.B. _Stephanie geht <u>aus</u>_), die oft auch Präpositionen sein können.

Um diese Hypothese näher zu untersuchen, können wir z.B. eine separate Evaluation nur für unbekannte Wörter durchführen. Wir erkennen diese daran, dass ihre Merkmalsvektoren $\mathbf{0}$ sind. Wir sehen nun, dass alle unbekannten Wörter als `NN` getaggt werden und dass es sich dabei überwiegend um offene Wortklassen handelt (Spalte _support_).

In [ ]:
idx_oov = testX_wf.sum(axis=1) == 0
idx_oov = np.asarray(idx_oov).squeeze() # Spaltenvektor (Typ: np.matrix) in Vektor konvertieren
print(classification_report(test.pos[idx_oov], predicted[idx_oov], zero_division=0))

Schließlich können wir noch eine Fehlermatrix (_confusion matrix_) berechnen, die die häufigsten Fehler aufzeigt und schön visualisiert werden kann. Da sich die volle Fehlermatrix nur schwer darstellen lässt, konzentrieren wir uns auf ausgewählte Wortarten, z.B. Substantive, Vollverben, Adjektive und Adverbien.

In [ ]:
focus_tags = ('ADJA', 'ADJD', 'ADV', 'NE', 'NN', 'VVFIN', 'VVINF', 'VVIMP')
cm = confusion_matrix(test.pos, predicted, labels=focus_tags)
ConfusionMatrixDisplay(cm, display_labels=focus_tags).plot(cmap='OrRd');

Auffallend ist eine Spalte, mit zahlreichen Fehlern, bei denen das Lernverfahren `NN` vorhergesagt hat. Hier handelt es sich mutmaßlich zu einem großen Teil um unbekannte Wörter. Im nächsten Schritt gilt es nun also, Wortarten für solche unbekannten Wörter zu erraten.

### Präfix- und Suffixmerkmale

Die Wortart eines unbekannten Wortes lässt sich am ehesten aus der Endung (Suffix der letzten $k$ Zeichen, z.B. _-bares_) erraten, sowie teilweise auch aus dem Wortanfang (z.B. _ge-_). Wie viele Zeichen $k$ zu berücksichtigen sind, kann nur empirisch durch Experimente mit verschiedenen Parametereinstellungen ermittelt werden.

Wir könnten nun zusätzliche Spalten mit den jeweiligen Präfixen und Suffixen in unseren Datentabellen ergänzen (evtl. auch zu Kleinschreibung normalisiert) und darauf einen `OneHotEncoder` anwenden.

In [ ]:
tmp = test.copy()
tmp['suff4'] = tmp.word.str.lower().str[-4:]
tmp

Hier nützen wir stattdessen wieder den `CountVectorizer` mit einer geeigneten Tokenizer-Funktion (die alle gewünschten Suffixe und Präfixe zurückliefert) und der voreingestellten Normalisierung zu Kleinschreibung. Wichtig ist, dass Präfix und Suffix der gleichen Länge unterschieden werden!

In [ ]:
def get_prefix_suffix(word):
    l = len(word)
    res = []
    for k in range(2, 5):
        if l > k:
            res.append("-" + word[-k:])
    for k in range(2, 4):
        if l > k:
            res.append(word[:k] + "-")
    return(res)

print(get_prefix_suffix(test.word[0]))
print(get_prefix_suffix(test.word[1]))

Sinnvoll sind vor allem Affixe, die häufig genug vorkommen, um zuverlässige Informationen zu liefern. Wir zeigen hier zunächst, welche Affixe bei $f\ge 500$ verwendet werden. Für die tatsächliche Merkmalsextraktion verwenden wir dann einen weitaus niedrigeren Schwellenwert (der auch von der Länge $k$ des Affix abhängig gemacht werden könnte).

In [ ]:
affix_vectorizer = CountVectorizer(tokenizer=get_prefix_suffix, min_df=500)
affix_vectorizer.fit(train.word)
print(" ".join(affix_vectorizer.get_feature_names_out()))

In [ ]:
affix_vectorizer = CountVectorizer(tokenizer=get_prefix_suffix, min_df=20)
X_affix = affix_vectorizer.fit_transform(train.word)
testX_affix = affix_vectorizer.transform(test.word)
X_affix.shape

Schließlich trainieren wir eine SVM mit der kombinierten Merkmalsmatrix.

In [ ]:
X = sp.sparse.hstack([X_wf, X_affix])
testX = sp.sparse.hstack([testX_wf, testX_affix])

In [ ]:
clf = LinearSVC()
%time clf.fit(X, train.pos)
%time clf.score(testX, test.pos)

### Weitere Merkmale

> **Aufgabe:** Extrahieren Sie spezifische Merkmale wie Groß-/Kleinschreibung, „Token besteht nur aus Ziffern“, „Token enthält keine Buchstaben“, „Bindestrich am Wortend“, Satzanfang, usw. als dicht besetzte Binärmatrix.  Denken Sie sich auch weitere Merkmale aus, die für die Wortartenerkennung nützlich sein könnten.  Fügen Sie die zusätzlichen Merkmale dann an die bisherige Merkmalsmatrix an.  Können Sie damit die Genauigkeit des Unigramm-Taggers verbessern?  

> **Frage:** Können Sie erklären, warum einige dieser Merkmale (z.B. die Markierung für den Satzanfang) von einem linearen Klassifikator (wie der hier verwendeten `LinearSVC`) nicht optimal genutzt werden? Wie könnte man angesichts dieser Erkenntnis die Ergebnisse möglicherweise noch etwas verbessern?

### Optimierung

> **Aufgabe:** Experimentieren Sie mit den Metaparametern des Lernverfahrens und der Merkmalsextraktion (z.B. OOV-Schwellenwert für Wortformen, maximale Länge der Präfixe und Suffixe, hinzufügen von weiteren spezifischen Merkmalen. Testen Sie auch andere Lernverfahren als SVM: können diese schneller trainiert werden? Denken Sie, dass eine systematische Optimierung der Metaparameter (insb. der Regularisierungsstärke) sinnvoll ist?